In [1]:
print("hello")

hello


In [1]:
from langchain_community.utilities import SQLDatabase
from langchain_community.llms import Ollama
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain.agents import create_sql_agent
from langchain.agents.agent_types import AgentType
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.chains import LLMChain
import os

In [2]:
def setup_sql_agent(db_file: str):
    db_uri=f"sqlite:///{db_file}"
    db=SQLDatabase.from_uri(db_uri)
    
    llm = Ollama(
        model="mistral:latest",
        temperature=0.1
    )
    toolkit=SQLDatabaseToolkit(db=db,llm=llm)
    agent=create_sql_agent(
        llm=llm,
        toolkit=toolkit,
        agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True
    )
    
    explanation_prompt=PromptTemplate(
        input_variables=["query","results"],
        template="""Given this SQL query :
        {query}
        and these resuts:
        {result}
        provide a clear, concise explanation of what these result mean in natural language.
        focus on the key insights and pattern in the data.
        """
    )
    
    explanation_chain=LLMChain(
        llm=llm,
        prompt=explanation_prompt,
        output_parser=StrOutputParser()
    )
    
    # return llm,db,toolkit,agent,explanation_chain
    return agent,db

def query_database(agent,question):
    
    try:
        result=agent.run(question)
        return result
    except Exception as e:
        return f"Error occured: {str(e)}"
    

def get_db_schema(db):
    return db.get_table_info()



if __name__ =="__main__":
    
    agent, db = setup_sql_agent("chinook.db")
    

    print("Database Schema:")
    print(get_db_schema(db))
    
    # Example questions you can ask about the chinook database
    questions = [
        "Find the most expensive track in the database",
        "Show the number of tracks available in each genre"
    ]
    
    #Run example questions
    for question in questions:
        print(f"\nQuestion: {question}")
        print("Answer:", query_database(agent, question))
    
    # question="How many employees are there?"
    # print(f"\nQuestion: {question}")
    # print("Answer:", query_database(agent, question))

C:\Users\NIAZ\AppData\Local\Temp\ipykernel_13196\3435108495.py:5: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(
C:\Users\NIAZ\AppData\Local\Temp\ipykernel_13196\3435108495.py:28: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  explanation_chain=LLMChain(
C:\Users\NIAZ\AppData\Local\Temp\ipykernel_13196\3435108495.py:40: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result=agent.run(question)


Database Schema:

CREATE TABLE albums (
	"AlbumId" INTEGER NOT NULL, 
	"Title" NVARCHAR(160) NOT NULL, 
	"ArtistId" INTEGER NOT NULL, 
	PRIMARY KEY ("AlbumId"), 
	FOREIGN KEY("ArtistId") REFERENCES artists ("ArtistId")
)

/*
3 rows from albums table:
AlbumId	Title	ArtistId
1	For Those About To Rock We Salute You	1
2	Balls to the Wall	2
3	Restless and Wild	2
*/


CREATE TABLE artists (
	"ArtistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("ArtistId")
)

/*
3 rows from artists table:
ArtistId	Name
1	AC/DC
2	Accept
3	Aerosmith
*/


CREATE TABLE customers (
	"CustomerId" INTEGER NOT NULL, 
	"FirstName" NVARCHAR(40) NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"Company" NVARCHAR(80), 
	"Address" NVARCHAR(70), 
	"City" NVARCHAR(40), 
	"State" NVARCHAR(40), 
	"Country" NVARCHAR(40), 
	"PostalCode" NVARCHAR(10), 
	"Phone" NVARCHAR(24), 
	"Fax" NVARCHAR(24), 
	"Email" NVARCHAR(60) NOT NULL, 
	"SupportRepId" INTEGER, 
	PRIMARY KEY ("CustomerId"), 
	FOREIGN KEY("SupportRepId") R